In [1]:
import numpy as np
import pandas as pd

In [2]:
import pandas as pd
import numpy as np

import sys
sys.path.insert(1, 'dnn/')

# del bnn
import bcell_nn as bnn
import bcell_plot
import bcell_preprocess as bpreprocess
# import bcell_driver

# System arguments
import argparse
# data management
import numpy as np
import pandas as pd
# visualization
import matplotlib.pyplot as plt
import seaborn as sns
# flow cytometry libraries
import cytoflow as flow
# user defined functions

from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12630782183149209809
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 8178714378970443712
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 4491425846570173769
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10719745792
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1238766978359951821
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


In [4]:
# Read in signal data

In [5]:
df1 = pd.read_csv('../data/Bcell_signalling_proteins/matrix_basal_signalling_markers_#2.csv', index_col=0)
df2 = pd.read_csv('../data/Bcell_signalling_proteins/matrix_bcr_signalling_markers_#2.csv', index_col=0)
df1['bcr'] = 0
df2['bcr'] = 1

signal_df = pd.concat([df1, df2])
signal_df.head()

print(signal_df.shape)

(29702, 23)


In [6]:
# read in surface data

In [7]:
df1 = pd.read_csv( '../data/Bcell_surface_marker/matrix_basal_surface_markers_#2.csv', index_col=0)
df2 = pd.read_csv( '../data/Bcell_surface_marker/matrix_bcr_surface_markers_#2.csv', index_col=0)

df1['bcr'] = 0
df2['bcr'] = 1

surface_df = pd.concat([df1, df2])
surface_df.head()
print(surface_df.shape)

(29702, 14)


# train

In [8]:
USE = 1
VAL = 0.2
TEST = 0.1

SHAPE = [10, 8, 4]

## Model that trains signalling

In [9]:
# Use all data
cyto_dataset = bpreprocess.df_to_train_tensor(signal_df, use=USE)
train_dataset, val_dataset, test_dataset = bpreprocess.split_dataset(cyto_dataset, VAL, TEST)

In [10]:
model = bnn.define_model(shape=SHAPE, dropout=0.1)
model, history = bnn.fit_model(model, train_dataset, val_dataset, train_dataset, batch_size=256, epochs=50, patience=10)
score, acc = model.evaluate(test_dataset)

Epoch 1/50

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

20868/20868 [==============================] - 68s 3ms/step - loss: 0.5665 - accuracy: 0.7385 - val_loss: 0.4177 - val_accuracy: 0.8120
Epoch 2/50
20868/20868 [==============================] - 67s 3ms/step - loss: 0.4233 - accuracy: 0.7846 - val_loss: 0.3847 - val_accuracy: 0.8253
Epoch 3/50
20868/20868 [==============================] - 67s 3ms/step - loss: 0.4032 - accuracy: 0.7962 - val_loss: 0.3676 - val_accuracy: 0.8277
Epoch 4/50
20868/20868 [==============================] - 66s 3ms/s

In [11]:
# Save model
model_filename = bnn.save_model(model, '../models/signalling_naive', history=history)
print("Model saved to: ", model_filename)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ../models/signalling_naive1/assets
Model saved to:  ../models/signalling_naive1


### Use only 152-Ki67, 169-pP38 and 171-pBtk/Itk

In [12]:
# Use all data
my_signal = signal_df.loc[:, ['152-Ki67', '169-pP38', '171-pBtk/Itk', 'bcr']]
cyto_dataset = bpreprocess.df_to_train_tensor(my_signal)
train_dataset, val_dataset, test_dataset = bpreprocess.split_dataset(cyto_dataset, VAL, TEST)

In [13]:
model2 = bnn.define_model(shape=SHAPE, dropout=0.1)
model2, history2 = bnn.fit_model(model2, train_dataset, val_dataset, train_dataset, batch_size=256, epochs=50, patience=10)
score2, acc2 = model2.evaluate(test_dataset)

Epoch 1/50

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

20868/20868 [==============================] - 65s 3ms/step - loss: 0.6983 - accuracy: 0.5276 - val_loss: 0.6916 - val_accuracy: 0.5278
Epoch 2/50
20868/20868 [==============================] - 65s 3ms/step - loss: 0.6958 - accuracy: 0.5240 - val_loss: 0.6931 - val_accuracy: 0.5166
Epoch 3/50
20868/20868 [==============================] - 64s 3ms/step - loss: 0.6916 - accuracy: 0.5299 - val_loss: 0.6915 - val_accuracy: 0.5265
Epoch 4/50
20868/20868 [==============================] - 64s 3ms/s

In [14]:
# Save model
model_filename = bnn.save_model(model2, '../models/signalling_specific', history=history2)
print("Model saved to: ", model_filename)

INFO:tensorflow:Assets written to: ../models/signalling_specific1/assets
Model saved to:  ../models/signalling_specific1


## Model that trains surface

In [15]:
# Use all data
cyto_dataset = bpreprocess.df_to_train_tensor(surface_df)
train_dataset, val_dataset, test_dataset = bpreprocess.split_dataset(cyto_dataset, VAL, TEST)

In [16]:
model = bnn.define_model(shape=SHAPE, dropout=0.1)
model, history = bnn.fit_model(model, train_dataset, val_dataset, train_dataset, batch_size=256, epochs=50, patience=10)
score, acc = model.evaluate(test_dataset)

Epoch 1/50

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

20868/20868 [==============================] - 65s 3ms/step - loss: 0.5962 - accuracy: 0.6976 - val_loss: 0.5317 - val_accuracy: 0.7332
Epoch 2/50
20868/20868 [==============================] - 65s 3ms/step - loss: 0.5395 - accuracy: 0.7146 - val_loss: 0.5123 - val_accuracy: 0.7513
Epoch 3/50
20868/20868 [==============================] - 64s 3ms/step - loss: 0.5365 - accuracy: 0.7265 - val_loss: 0.5143 - val_accuracy: 0.7684
Epoch 4/50
20868/20868 [==============================] - 64s 3ms/s

In [17]:
# Save model
model_filename = bnn.save_model(model, '../models/surface_naive', history=history)
print("Model saved to: ", model_filename)

INFO:tensorflow:Assets written to: ../models/surface_naive1/assets
Model saved to:  ../models/surface_naive1


### Use only 152-Ki67, 169-pP38 and 171-pBtk/Itk

In [18]:
# Use all data 
my_surface = surface_df.loc[:, ['167-CD38', '142-CD19', '147-CD20', '148-CD34', 'bcr']]
cyto_dataset = bpreprocess.df_to_train_tensor(my_surface)
train_dataset, val_dataset, test_dataset = bpreprocess.split_dataset(cyto_dataset, VAL, TEST)

In [19]:
model2 = bnn.define_model(shape=SHAPE, dropout=0.1)
model2, history2 = bnn.fit_model(model2, train_dataset, val_dataset, train_dataset, batch_size=256, epochs=50, patience=10)
score2, acc2 = model2.evaluate(test_dataset)

Epoch 1/50

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

20868/20868 [==============================] - 65s 3ms/step - loss: 0.7030 - accuracy: 0.5275 - val_loss: 0.6920 - val_accuracy: 0.5182
Epoch 2/50
20868/20868 [==============================] - 64s 3ms/step - loss: 0.6930 - accuracy: 0.5265 - val_loss: 0.6969 - val_accuracy: 0.5251
Epoch 3/50
20868/20868 [==============================] - 64s 3ms/step - loss: 0.7341 - accuracy: 0.5249 - val_loss: 0.6924 - val_accuracy: 0.5278
Epoch 4/50
20868/20868 [==============================] - 64s 3ms/s

In [20]:
# Save model
model_filename = bnn.save_model(model2, '../models/surface_specific', history=history2)
print("Model saved to: ", model_filename)

INFO:tensorflow:Assets written to: ../models/surface_specific1/assets
Model saved to:  ../models/surface_specific1
